K-neighbors es el modelo mas adecuado para este sistema de recomendaciones. Este modelo puede definir istintas medidas de similaridad como distancia euclidiana o cosine similarity.

#Tranformar el dataset

In [14]:
#importar las librerias y leer el archivo
import pandas as pd

games = pd.read_csv('steam_games.csv')

In [15]:
#hacerle cambios a la columna de release date para que quede con formato de fecha, 
#crear una columna de año y rellenar losvalores faltantes 
games['release_date'] = pd.to_datetime(games['release_date'], errors='coerce')
games['release_year'] = games['release_date'].dt.year
games['release_year'] = games['release_year'].fillna(0).astype(int)
games['release_year'] = games['release_year'].astype('str').replace('0', '')

In [16]:
games.fillna('', inplace=True)

games.drop(games[games['title'] == ""].index, inplace=True)
games.shape

(30085, 14)

In [17]:
# limpiar la columna developer para que no tenga contenga caracteres inusuales'
games['developer'] = games['developer'].str.replace(r'\s+', '-', regex=True)
games['developer'] = games['developer'].str.replace(r'---', "-")

In [18]:
import re
import ast

# función de limpieza de las columnas
#la primera parte del codigo funciona para volver a forma de lista un conjunto de datos, esto se usara en las columnas anidadas
def tolist(lst):
    text=""
    if isinstance(lst, str) and len(lst) > 0 and '[' in lst:
        lst = lst.lower()
        lst = lst.replace("0's",'0s').replace(';','').replace('&','').replace("'em","em")
        try:
            lst =  ast.literal_eval(lst)
        except (SyntaxError, ValueError):
            return lst.lower()
        
    if isinstance(lst,list):
        for item in lst:
            while "  " in item:
                item = item.replace("  "," ")
            item = item.replace(" ", "-")
            text = text + " " + item 
            text = text.replace('"','')
        return text.strip().lower()

    return lst

In [19]:
# aplicamos la funcion a cada columna anidada
games['specs'] = games['specs'].apply(lambda x : tolist(x))
games['genres'] = games['genres'].apply(lambda x : tolist(x))
games['tags'] = games['tags'].apply(lambda x : tolist(x))

In [20]:
# Combine relevant columns into a single 'profile'
games['profile'] = games['genres'] + " " + games['tags'] + " " + games['specs'] + " " + games['developer'] + " " + games['release_year']

# Dataset listo para usarse en el modelo
games[['id','title','profile']].to_csv("ml_dataset.csv",index=False)

#aplicar el modelo K-neighbors

In [21]:
#importar la librerias para trabajar el modelo, es importante la libreria de vectorizar para hacer vectores
#con las palabras y la libreria del modelo K-neighbors
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

df = pd.read_csv("ml_dataset.csv")

# se crea un vector con todos los datos de la columna profile para que se usen en el modelo
tfidf = TfidfVectorizer(stop_words='english',max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['profile'])

# usar la libreria K-neighbors para encontrar los uegos similares
knn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')  # Se coloca 6 pues tiene que considerar el juego del que se tiene referencia
knn.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [22]:
# Crear función que crea las recomendaciones
def recommend(game_id):
    #agrega los indices de juegos iguales al game_id y devuelve el primero que encuentre
    try:
        idx = df.index[df['id'] == game_id].tolist()[0]
        #obitnene el nombre del juego con el indice encontrado en la linea superior
        name = df['title'].loc[idx]
        
        values , indices = knn.kneighbors(tfidf_matrix[idx])
        
        # se excluye el primer valor pues es el id que se puso en la funcion
        game_indices = indices[0][1:]
        result = df['id'].iloc[game_indices]
    except:
        return [0]
    return list(result)

In [23]:
# test
recommend(47810)

[47730.0, 17450.0, 17460.0, 7110.0, 24980.0]

In [24]:
# un df sin la columna profile
df1 = df[['id','title']]

In [25]:
# función que crea la columna con las recomendaciones de cada juego
df1['recommends'] = df1['id'].apply(lambda x  : recommend(x))

In [26]:
# Dataset para usar en la API listo 
df1['id'].fillna(0,inplace=True)
df1.to_csv("modelo.csv",index=False)
df1.info()

C:\Users\Public\Documents\iSkysoft\CreatorTemp\ipykernel_18704\52715459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['id'].fillna(0,inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30085 entries, 0 to 30084
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          30085 non-null  float64
 1   title       30085 non-null  object 
 2   recommends  30085 non-null  object 
dtypes: float64(1), object(2)
memory usage: 705.2+ KB


In [27]:
df1

,id,title,recommends
0,761140.0,Lost Summoner Kitty,"[765320.0, 766850.0, 697000.0, 730120.0, 55361..."
1,643980.0,Ironbound,"[544730.0, 205890.0, 453480.0, 410380.0, 68542..."
2,670290.0,Real Pool 3D - Poolians,"[522660.0, 645260.0, 614100.0, 504350.0, 52442..."
3,767400.0,弹炸人2222,"[767400.0, 686070.0, 562690.0, 670100.0, 70863..."
4,772540.0,Battle Royale Trainer,"[302250.0, 434250.0, 3700.0, 34830.0, 238090.0]"
...,...,...,...
30080,745400.0,Kebab it Up!,"[772190.0, 766700.0, 718050.0, 680120.0, 71859..."
30081,773640.0,Colony On Mars,"[758480.0, 745500.0, 745510.0, 761440.0, 75847..."
30082,733530.0,LOGistICAL: South Africa,"[706380.0, 718490.0, 693650.0, 706390.0, 73352..."
30083,610660.0,Russian Roads,"[754510.0, 701720.0, 754520.0, 735410.0, 67366..."


Funcion de recomendacion

In [30]:
# Función del endpoint
import pandas as pd
import ast

def recomendacion_juego( game_id : int ):
    # carga de archivos
    df = pd.read_csv("modelo.csv")

    if game_id not in df['id'].values:
        return 'error'

    # Obtiene la lista de recomendaciones
    result = df[df['id'] == game_id]['recommends'].iloc[0]

    try:
        result = ast.literal_eval(result)
    except (SyntaxError, ValueError):
        print('No se puede convertir a lista')
        return 'error'

    response =[]
    item_buscado = df[df['id'] == game_id].iloc[0]
    
    for item_id in result:
        # Obtiene la información del juego recomendado
        item_info = df[df['id'] == item_id].iloc[0]  

        response.append({'game_id': int(item_info['id']), 
                         'title': item_info['title']})
    dict_respuesta = {'titulo_buscado': item_buscado['title'], 'results':response}  

    return dict_respuesta

In [31]:
# test
recomendacion_juego(47810)

{'titulo_buscado': 'Dragon Age: Origins - Ultimate Edition',
 'results': [{'game_id': 47730, 'title': 'Dragon Age™: Origins Awakening'},
  {'game_id': 17450, 'title': 'Dragon Age: Origins'},
  {'game_id': 17460, 'title': 'Mass Effect'},
  {'game_id': 7110, 'title': 'Jade Empire™: Special Edition'},
  {'game_id': 24980, 'title': 'Mass Effect 2'}]}